In [3]:
import pandas as pd


In [4]:
df_train = pd.read_csv('data/fraudTrain.csv')
df_train.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

Determinting which columns are neccessary for prediction

In [4]:
df_train.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
df_train['trans_date_trans_time'] =pd.to_datetime(df_train['trans_date_trans_time'], format='%Y-%m-%d %H:%M:%S')

In [6]:
df_train['trans_time'] = df_train['trans_date_trans_time'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

In [9]:
df_train.drop(columns=['trans_date_trans_time', "unix_time"], inplace=True)

In [7]:
df_train['category'].unique()

array(['misc_net', 'grocery_pos', 'entertainment', 'gas_transport',
       'misc_pos', 'grocery_net', 'shopping_net', 'shopping_pos',
       'food_dining', 'personal_care', 'health_fitness', 'travel',
       'kids_pets', 'home'], dtype=object)

In [21]:
df_train.columns

Index(['category', 'amt', 'lat', 'long', 'job', 'trans_num', 'merch_lat',
       'merch_long', 'is_fraud', 'trans_time'],
      dtype='object')

An ideal model will identify fradulent transactions without using the credit card number (cc_num) because not all transactions could be fradulent. Street, City, State, first name, last name, Date of birth and Gender should not be used for training the model for privacy reasons. City Population is not neccessary feature. For now, we exclude Merchant Name. Transaction number is not feature. Job can also be used to identify credit card frauds because the probability of high-paying job person is doing fradulent transaction is very less. But for now, we will exclude jobs 

In [12]:
df_train.drop(columns=['street', 'city', 'state', 'first', 'last', 'dob', 'zip', 'city_pop'], inplace=True)

In [16]:
df_train.drop(columns=['merchant', 'cc_num'], inplace=True)

In [20]:
df_train.drop(columns=['gender'], inplace=True)

In [11]:
df_train['category'].value_counts()

category
gas_transport     131659
grocery_pos       123638
home              123115
shopping_pos      116672
kids_pets         113035
shopping_net       97543
entertainment      94014
food_dining        91461
personal_care      90758
health_fitness     85879
misc_pos           79655
misc_net           63287
grocery_net        45452
travel             40507
Name: count, dtype: int64

In [18]:
df_train['job'].value_counts()

job
Film/video editor             9779
Exhibition designer           9199
Naval architect               8684
Surveyor, land/geomatics      8680
Materials engineer            8270
                              ... 
Veterinary surgeon               8
Information officer              8
Contracting civil engineer       7
Ship broker                      7
Warehouse manager                7
Name: count, Length: 494, dtype: int64

In [5]:
df_train['job'][df_train['is_fraud']==1].value_counts()

job
Materials engineer                      62
Trading standards officer               56
Naval architect                         53
Exhibition designer                     51
Surveyor, land/geomatics                50
                                        ..
Chartered loss adjuster                  3
Statistician                             3
Health physicist                         3
Contractor                               2
English as a second language teacher     2
Name: count, Length: 443, dtype: int64

In [23]:
df_train['amt'][df_train['is_fraud']==1].describe()

count    7506.000000
mean      531.320092
std       390.560070
min         1.060000
25%       245.662500
50%       396.505000
75%       900.875000
max      1376.040000
Name: amt, dtype: float64

In [5]:
from sklearn.preprocessing import OneHotEncoder

In [5]:
from utils import preprocess_data

df_train = preprocess_data(df_train)
df_train.columns

Index(['amt', 'lat', 'long', 'trans_num', 'merch_lat', 'merch_long',
       'is_fraud', 'trans_time', 'category_food_dining',
       'category_gas_transport', 'category_grocery_net',
       'category_grocery_pos', 'category_health_fitness', 'category_home',
       'category_kids_pets', 'category_misc_net', 'category_misc_pos',
       'category_personal_care', 'category_shopping_net',
       'category_shopping_pos', 'category_travel'],
      dtype='object')

In [7]:
df_train[['amt', 'trans_time', 'is_fraud']].head()

,amt,trans_time,is_fraud
0,4.97,18,0
1,107.23,44,0
2,220.11,51,0
3,45.00,76,0
4,41.96,186,0
